# Calculating the Relationship between Agricultural Prices and Vegetation Health

##CURRENTLY CREATING A PANDAS DATAFRAME BY SCRAPING THE HTML ON THE WEBSITE

The purpose of this notebook is to advance and develop my own skills in terms of data analysis, data visualization, and coding adeptability, by creating and answering a relevant research question. Due to my own interest, this project will focus on predicting the price of avocados based on environmental characteristics. It will involve finding a dataset (from Kaggle) and importing this as the dependent variable. A representative number of agricultural regions will then need to be found (GGE appears to have some stuff that is relevant) and then historical NDVI calculated. Will also need to account for lag effect.

Data for historical avocado prices is available from http://www.hassavocadoboard.com/retail/volume-and-price-data. Will need to firstly collect and collate all avocado prices.

In [59]:
##Create the raw data files for each year of avocado data. No need to run if already done.
##Due to issues with the excel tables when downloaded, HTML tables were scraped instead.

from bs4 import BeautifulSoup
import requests
import pandas as pd

startyear = 2004
endyear = 2017

del df
del full_df
for year in range(startyear, endyear+1):
    
    #import_location = "Data/Raw/avocado_" + str(year) + ".csv"  #If saving data for later
    if year > 2014:
        price_page = requests.get("http://www.hassavocadoboard.com/retail/volume-and-price-data/historical-volume-and-price-data/" + str(year)).text
        soup = BeautifulSoup(price_page,"html.parser")
        
        ##Find the subsection of the page focussing on total avocado sales etc. in the US, not on the states. Returns array, first element has attributes.
        total_soup = soup.find_all(id = "rvpTotalUS")
        
        ##There are a number of additional table features we don't want (two to be exact, which appear before the header and data sections).
        ##Filter so that only table elements which don't have a class are returned.
        table = total_soup[0].find_all('table', {'class': None}) # Grab the first table
        ordered_table = []
        for row in table:
            this_row = row.find_all("td")
            new_row = [data.text for data in this_row]
            ordered_table.append(new_row)
        df = pd.DataFrame(ordered_table)
        #Correcting header values
        head_vals = df.iloc[0]
        df = df[1:]
        df.columns = head_vals
        df.rename(columns={'ASP*':'Price'}, inplace=True)


    else:
        price_page = requests.get("http://www.hassavocadoboard.com/retail/volume-and-price-data/historical-volume-and-price-data/" + str(year)).text
        soup = BeautifulSoup(price_page,"html.parser")
        table = soup.find_all('tr')
        ordered_table = []
        for row in table:
            this_row = row.find_all('td')
            if this_row == []:
                this_row = row.find_all('th') #Tables are seperated into header and data
            new_row= [data.text for data in this_row]
            ordered_table.append(new_row)
        df = pd.DataFrame(ordered_table)
        #Correcting header values
        head_vals = df.iloc[0]
        df = df[1:]
        df.columns = head_vals
        df.rename(columns={'Average Price':'Price'}, inplace=True) #Align column names with other data format
        df.rename(columns={'Bags':'Total Bagged'}, inplace=True)
    
    #append data)
    if 'full_df' in globals():
        full_df = full_df.append(df, ignore_index = True, sort = False)
    else:
        full_df = df
    
df = full_df

In [ ]:
##Alter the date formats, specify months and years as we will want to work with these.
##Correct formats.

In [62]:
##Save the df if you wish
df.to_csv("Data/avocado_prices.csv")